In [10]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [11]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

print(sys.path)


['/Users/gali/anaconda3/envs/LIMEtf/lib/python310.zip', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/lib-dynload', '', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages', '/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/setuptools/_vendor', '../', '../', '../']


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [12]:
x_train, y_train,x_test_original, y_test_original = ReadTS('MiddlePhalanxTW')
y_train = y_train - 3
y_test_original = y_test_original - 3
#print(x_train.shape)
#print(y_train)

In [13]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):
    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train)

In [14]:
model = make_CNN_model(x_train.shape[1:],num_classes = 6)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
10/10 [==============================] - 1s 19ms/step - loss: 1.5541 - sparse_categorical_accuracy: 0.4295 - val_loss: 1.7801 - val_sparse_categorical_accuracy: 0.2750
Epoch 2/100
10/10 [==============================] - 0s 11ms/step - loss: 1.3181 - sparse_categorical_accuracy: 0.5392 - val_loss: 1.7192 - val_sparse_categorical_accuracy: 0.4250
Epoch 3/100
10/10 [==============================] - 0s 10ms/step - loss: 1.2283 - sparse_categorical_accuracy: 0.5517 - val_loss: 1.6615 - val_sparse_categorical_accuracy: 0.3750
Epoch 4/100
10/10 [==============================] - 0s 10ms/step - loss: 1.1776 - sparse_categorical_accuracy: 0.5925 - val_loss: 1.6265 - val_sparse_categorical_accuracy: 0.3750
Epoch 5/100
10/10 [==============================] - 0s 10ms/step - loss: 1.1249 - sparse_categorical_accuracy: 0.6238 - val_loss: 1.6103 - val_sparse_categorical_accuracy: 0.3750
Epoch 6/100
10/10 [==============================] - 0s 10ms/step - loss: 1.0796 - sparse_categorica

In [15]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

5/5 [==============================] - 0s 2ms/step - loss: 1.5981 - sparse_categorical_accuracy: 0.4351
Test accuracy 0.4350649416446686
Test loss 1.5980825424194336


In [16]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    print(average_l2_norm)

    return average_l2_norm


perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 30
index_1 = 80

def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [18]:
strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'MiddlePhalanxTW.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'MiddlePhalanxTWl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

5/5 [==============================] - 0s 3ms/step
6.622172083897745
5/5 [==============================] - 0s 2ms/step
6.974083959769382
5/5 [==============================] - 0s 2ms/step
6.865684654800217
5/5 [==============================] - 0s 2ms/step
10.6393908304404
5/5 [==============================] - 0s 2ms/step
1.6847170080406377


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

5/5 [==============================] - 0s 3ms/step
done
   Strategy  Accuracy  Precision    Recall  F1 Score
0  original  0.435065   0.488125  0.435065  0.447326
1       RBP  0.344156   0.413526  0.344156  0.335716
2      MRBP  0.077922   0.277896  0.077922  0.022326
3      zero  0.428571   0.425486  0.428571  0.408227
4     noise  0.292208   0.233464  0.292208  0.195806
5      blur  0.363636   0.465473  0.363636  0.395744
     L2-Norm
0   6.622172
1   6.974084
2   6.865685
3  10.639391
4   1.684717


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
